# `mnist.py`

In [2]:
# An MNIST loader.

import numpy as np
import gzip
import struct


def load_images(filename):
    # Open and unzip the file of images:
    with gzip.open(filename, 'rb') as f:
        # Read the header information into a bunch of variables
        _ignored, n_images, columns, rows = struct.unpack('>IIII', f.read(16))
        # Read all the pixels into a NumPy array of bytes:
        all_pixels = np.frombuffer(f.read(), dtype=np.uint8)
        # Reshape the pixels into a matrix where each line is an image:
        return all_pixels.reshape(n_images, columns * rows)


def prepend_bias(X):
    # Insert a column of 1s in the position 0 of X.
    # (“axis=1” stands for: “insert a column, not a row”)
    return np.insert(X, 0, 1, axis=1)


# 60000 images, each 785 elements (1 bias + 28 * 28 pixels)
X_train = prepend_bias(load_images("input/train-images.idx3-ubyte.gz"))

# 10000 images, each 785 elements, with the same structure as X_train
X_test = prepend_bias(load_images("input/t10k-images.idx3-ubyte.gz"))


def load_labels(filename):
    # Open and unzip the file of images:
    with gzip.open(filename, 'rb') as f:
        # Skip the header bytes:
        f.read(8)
        # Read all the labels into a list:
        all_labels = f.read()
        # Reshape the list of labels into a one-column matrix:
        return np.frombuffer(all_labels, dtype=np.uint8).reshape(-1, 1)


def encode_digit(Y, digit):
    encoded_Y = np.zeros_like(Y)
    n_labels = Y.shape[0]
    for i in range(n_labels):
        if Y[i] == digit:
            encoded_Y[i][0] = 1
    return encoded_Y


TRAINING_LABELS = load_labels("input/train-labels.idx1-ubyte.gz")
TEST_LABELS = load_labels("input/t10k-labels.idx1-ubyte.gz")

Y_train = []
Y_test = []

for digit in range(10):
    Y_train.append(encode_digit(TRAINING_LABELS, digit))
    Y_test.append(encode_digit(TEST_LABELS, digit))

# `main.py`

In [6]:
# A binary classifier that recognizes one of the digits in MNIST.

import numpy as np

# Applying Logistic Regression
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# Basically doing prediction but named forward as its 
# performing Forward-Propagation
def forward(X, w):
    weighted_sum = np.matmul(X, w)
    return sigmoid(weighted_sum)

# Calling the predict() function
def classify(X, w):
    return np.round(forward(X, w))


# Computing Loss over using logistic regression
def loss(X, Y, w):
    y_hat = forward(X, w)
    first_term = Y * np.log(y_hat)
    second_term = (1 - Y) * np.log(1 - y_hat)
    return -np.average(first_term + second_term)


# calculating gradient
def gradient(X, Y, w):
    return np.matmul(X.T, (forward(X, w) - Y)) / X.shape[0]

# calling the training function for desired no. of iterations
def train(X, Y, iterations, lr):
    w = np.zeros((X.shape[1], 1))
    for i in range(iterations):
        if i == 0 or i == 50 or i == iterations - 1:
            print('Iteration %4d => Loss: %.20f' % (i, loss(X, Y, w)))
        w -= gradient(X, Y, w) * lr
    return w

# Doing inference to test our model
def test(X, Y, w, digit):
    total_examples = X.shape[0]
    correct_results = np.sum(classify(X, w) == Y)
    success_percent = correct_results * 100 / total_examples
    print("Correct classifications for digit %d: %d/%d (%.2f%%)" %
          (digit, correct_results, total_examples, success_percent))

for digit in range(10):
    w = train(X_train, Y_train[digit], iterations=100, lr=1e-5)
    test(X_test, Y_test[digit], w, digit)

Iteration    0 => Loss: 0.69314718055994528623
Iteration   50 => Loss: 0.05717257769170278059
Iteration   99 => Loss: 0.04818443983574007689
Correct classifications for digit 0: 9899/10000 (98.99%)
Iteration    0 => Loss: 0.69314718055994528623
Iteration   50 => Loss: 0.04907664716890068612
Iteration   99 => Loss: 0.04270086047894513376
Correct classifications for digit 1: 9903/10000 (99.03%)
Iteration    0 => Loss: 0.69314718055994528623
Iteration   50 => Loss: 0.10590272802562278320
Iteration   99 => Loss: 0.09450611475477797840
Correct classifications for digit 2: 9737/10000 (97.37%)
Iteration    0 => Loss: 0.69314718055994528623
Iteration   50 => Loss: 0.11989407489302174314
Iteration   99 => Loss: 0.10998321696402391101
Correct classifications for digit 3: 9698/10000 (96.98%)
Iteration    0 => Loss: 0.69314718055994528623
Iteration   50 => Loss: 0.08956748403271110048
Iteration   99 => Loss: 0.07723638000157202754
Correct classifications for digit 4: 9759/10000 (97.59%)
Iteration 